In [1]:
import psycopg2 as pg
import pandas as pd

# Database setup
host = "localhost"
database = "cdm"
user = "postgres"
password = %env PGPASSWORD
connection_string = "host={} dbname={} user={} password={}".format(host, database, user, password)

db = pg.connect(connection_string)

## Diabetes and Differential Privacy

We have shown that single column counts can be perturbed to L1 norms.
Now we will be looking at diabetes as a case study for the usefulness of differential privacy. We will be exploring the effects of perturbing results 

In [2]:
#Retrieve the age and counts of the age of the patients who have the concept diabetes appear in their record
diabetics_query = """
    SELECT ages.age, COUNT(*)
    FROM (
    SELECT 
        p.person_id,
        min((EXTRACT (YEAR from con.condition_start_date)) - p.year_of_birth) as age
    FROM concept c, person p, condition_occurrence con
    WHERE
        con.person_id=p.person_id AND
        con.condition_concept_id=c.concept_id AND
        (c.concept_name LIKE '%Diabetes%' OR
        c.concept_name LIKE '%diabetes%')
    GROUP BY
        p.person_id) as ages
    GROUP BY ages.age;
    
"""
diabetics = pd.read_sql(diabetics_query, con=db)

In [ ]:
not_diabetics_query = """
    SELECT ages.age, count(*)
    FROM (SELECT 
        p.person_id,
        min((EXTRACT (YEAR from con.condition_start_date)) - p.year_of_birth) as age
    FROM concept c, person p, condition_occurrence con
    WHERE
        (con.person_id=p.person_id AND
        con.condition_concept_id=c.concept_id) AND p.person_id NOT IN
    """ + """(SELECT p.person_id
    FROM concept c, person p, condition_occurrence con
    WHERE
        con.person_id=p.person_id AND
        con.condition_concept_id=c.concept_id AND
        (c.concept_name LIKE '%Diabetes%' OR
        c.concept_name LIKE '%diabetes%')
        )
    GROUP BY
        p.person_id) as ages
    GROUP BY ages.age;
"""

#collects the age and counts of ages of 
non_diabetics = pd.read_sql(not_diabetics_query, con=db)

In [65]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go

# This is to use ployly offline
init_notebook_mode(connected=True)

iframe = None

#Original data
diab_data = go.Bar(x=diabetics['age'], y=diabetics['count'], name="diabetics", marker=dict(
        color='#FFA500'), opacity=0.75)
non_diab_data = go.Bar(x=non_diabetics['age'], y=non_diabetics['count'], name="non-diabetics", marker=dict(
        color='#0000FF'), opacity=0.75)

layout = go.Layout(
    title='Age Distribution of Diabetics vs Non-Diabetics', 
    xaxis={'title':'Age', 'tickangle': 300, 'exponentformat': 'none'}, 
    yaxis={'title':'Occurences'},
    showlegend=True,
    bargap=0.1,
    barmode='overlay')

data_all = [diab_data, non_diab_data]

fig = go.Figure(data=data_all, layout=layout)

iplot(fig, filename='overlaid histogram')

Now that we've shown the distributions for the top 10,000 diabetic and non-diabetic raw data, lets look at the differenitally private versions of these distributions

## Differentially Private Diabetics

In [73]:
for k,v in zip(diabetics['age'], diabetics['count']):
    k,v

In [57]:

#Differenially Private Data
diab_diff_private_data = go.Histogram(x=dp_diabetics['min'], name="DP_diabetics", marker=dict(
        color='#FFA500'), opacity=0.75)
non_diab_diff_private_data = go.Histogram(x=dp_non_diabetics['min'], name="DP_non_diabetics", marker=dict(
        color='#FFA500'), opacity=0.75)

NameError: name 'dp_diabetics' is not defined